# 2.1 primitive tensor function

机器学习编译过程 ---> 张量函数间的变换 \
不同的抽象能表示同一张量函数

```python
torch.add(a, b, c)
```
```python
def add(a, b, c):
    for i in range(128):
        c[i] = a[i] + b[i]
```
```C
void add(float* a, float* b, float* c){
    for(int i = 0; i < 128; i++){
        c[i] = a[i] + b[i];
    }
}
```

MLC: 将张量函数变换为更加专门的、针对特定工作和部署环境的函数
```python
for x in range(128):
    c[x] = a[x] + b[x]
```
串行 ------> 向量化
```python
parallel for xo in range(32):
    c[xo * 4: xo * 4 + 4] = f32_4.add(
        a[xo * 4: xo * 4 + 4], b[xo * 4: xo * 4 + 4])
```

# 2.2 张量程序抽象

抽象指定“做什么”，实现提供“如何”做; \
元张量函数组成：存储数据多维度数组、驱动计算的循环嵌套、计算部分本身语句

In [2]:
from tvm.script import tir as T

@T.prim_func
def main(A: T.Buffer[128, 'float32'],
         B: T.Buffer[128, 'float32'],
         C: T.Buffer[128, 'float32']):        # (Multi-dimensional) buffers -> input & output & intermediate results
    for i in range(128):                      # Loop nests -> drive compute iterations
        with T.block("C"):
            vi = T.axis.spatial(128, i)
            C[vi] = A[vi] + B[vi]             # Computations -> statements

张量程序抽象的一个重要性质是，他们能够被一系列有效的程序变换所改变 \


```python
# initial state
for x in range(128):
    C[x] = A[x] + B[x]
```
program-based transformations
1. xo, xi = split(x, 4)
2. parallel(xo)
3. vectorize(xi) \
------------------------------>
```python 
# Transformed program
parallel for xo in range(32):
    C[xo*4:xo*4+4] = f32_4.add(
        A[xo*4:xo*4+4], B[xo*4:xo*4+4])
```

## 2.2.1 张量程序抽象中的其他结构

一些计算会依赖于循环之间的顺序，不能随意变换。张量程序可以将这额额外的信息合并为程序的一部分，使程序变换更加便利
```python
from tvm.script import tir as T
@T.prim_func
def main(A: T.Buffer[128, 'float32'],
         B: T.Buffer[128, 'float32'],
         C: T.Buffer[128, 'float32']):
    for i in range(128):
        with T.block("C"):
            vi = T.axis.spatial(128, i)
            # Extra infomation about iteration
            # vi correspond to an interation of length 128 and can be spatially parallelized without dependency across other loop values of vi
            C[vi] = A[vi] + B[vi]
```

T.axis.spatial -> 表明 vi 这个特定的变量被映射到循环变量 i，并且所有的迭代都是独立的，可以安全地并行或者重新排序所有与 vi 有关的循环